# Dance of the bees
Batchrunner Data

Door Roeland Oostdam?, Thijme de Bruijn, Gerrit van de Bunt

ABD Challenge

# Gebruiksaanwijzing;
1. **Geef het absolute pad op naar de folder waar de folder 'IntelligentBeesChallenge' zich in bevindt.**
2. Stel parameters op voor de simulatie, deze kan je onderverdelen in variable en fixed parameters die je hier ook onder kan definieren. **Let op: voor variable parameters moet je een range object opgeven**
3. Zorg ervoor dat alle parameters zijn ingedeeld in of variable of fixed parameters en run dan alle cells.

# Parameters:
"height" : Hoogte van de grid van de omgeving \
"width : Breedte van de grid van de omgeving \
"init_bees" : Aantal bijen in de omgeving aan het begin \
"init_flowers" : Aantal bloemvelden in de omgeving aan het begin \
"min_nectar" : Minimale hoeveelheid nectar beschikbaar in bloemvelden \
"max_nectar" : Maximale hoeveelheid nectar beschikbaar in bloemvelden \
"init_min_nectar_grade" : Minimale kwaliteit nectar wat kan voorkomen \
"init_max_nectar_grade" : Maximale kwaliteit nectar wat kan voorkome \
"nectar_respawn_interval" : Aantal timesteps dat minimaal verstrijkt voordat nectar terug komt op een bloemenveld \
"max_bee_energy" : Capaciteit energie die een bij kan hebben nadat hij bij een bijenkorf geweest is en aan het begin \
"collect_negative_value_nectar" : Collect negative value nectar (?) \
"perception_range" : Hoe ver de bij kan zien om bloemenvelden/nectar te identificeren en op te slaan in memory.

In [1]:
# BATCHRUNNER PARAMS

# Voorbeeld:
# Als dit het pad is waar de notebook zich in bevindt;
# C:\Users\gvand\Projecten\Pycharm\ABD\IntelligentBeesChallenge\Dance of the bees Batchrunner Data.ipynb
# Geef dan dit op als project path:
# proj_path = r"C:\Users\gvand\Projecten\Pycharm\ABD"

proj_path = r"C:\Users\famil\PycharmProjects\pythonProject"

fixed_params = {
    "height": 10,
    "width": 10,
    "init_bees": 1,
    "init_flowers": 6,
    "init_min_nectar_grade": 1,
    "init_max_nectar_grade": 30,
    "min_nectar": 1,
    "max_nectar": 1,
    "nectar_respawn_interval": 50,
    "collect_negative_value_nectar": True,
    "perception_range": 1,
}

variable_params = {
    "max_bee_energy": range(30, 50, 10)
}

In [2]:
# Zet content root op om import problemen te voorkomen
import sys
sys.path.insert(1, proj_path)

In [3]:
# Importeer benodigde onderdelen voor de batchrunner
from mesa.batchrunner import BatchRunner, BatchRunnerMP
from IntelligentBeesChallenge.bee_simulation.model import BeeSimulation, get_nectar_per_t, get_bee_energy, get_hive_energy, \
    get_nectar_collected

In [4]:
batch_run = BatchRunner(BeeSimulation,
                        variable_parameters=variable_params,
                        fixed_parameters=fixed_params,
                        iterations=3,
                        model_reporters={
                            "Nectar/T": get_nectar_per_t,
                            "Bee energy": get_bee_energy,
                            "Nectar stored": get_hive_energy,
                            "Nectar Collected": get_nectar_collected,
                        })
batch_run.run_all()
model_data = batch_run.get_model_vars_dataframe()

0it [00:00, ?it/s]

Current State: explore
[[   -8.   -10.   -12.   -14.   -16.   -18.   -20.   -22.   -24.   -26.]
 [   -6.    -8.   -10.   -12.   -14.   -16.   -18.   -20.   -22.   -24.]
 [   -4.    -6.    -8.   -10.   -12.   -14.   -16.   -18.   -20.   -22.]
 [-1000.    -4.    -6.    -8.   -10.   -12.   -14.   -16.   -18.   -20.]
 [-1000.    14.    -4.    -6.    -8.   -10.   -12.   -14.   -16.   -18.]
 [-1000.    -4.    -6.    -8.   -10.   -12.   -14.   -16.   -18.   -20.]
 [   -4.    -6.    -8.   -10.   -12.   -14.   -16.   -18.   -20.   -22.]
 [   -6.    -8.   -10.   -12.   -14.   -16.   -18.   -20.   -22.   -24.]
 [   -8.   -10.   -12.   -14.   -16.   -18.   -20.   -22.   -24.   -26.]
 [  -10.   -12.   -14.   -16.   -18.   -20.   -22.   -24.   -26.   -28.]]
Current State: return_to_hive
Current State: explore
[[   -8.   -10.   -12.   -14.   -16.   -18.   -20.   -22.   -24.   -26.]
 [   -6.    -8.   -10.   -12.   -14.   -16.   -18.   -20.   -22.   -24.]
 [   -4.    -6.    -8.   -10.   -12.   -14.   -

1it [00:00,  3.95it/s]


Current State: explore
[[  -10.   -12.   -14.   -16.   -18.   -20.   -22.   -24.   -26.   -28.]
 [   -8.   -10.   -12.   -14.   -16.   -18.   -20.   -22.   -24.   -26.]
 [   -6.    -8.   -10.   -12.   -14.   -16.   -18.   -20.   -22.   -24.]
 [-1000.    -6.    -8.   -10.   -12.   -14.   -16.   -18.   -20.   -22.]
 [-1000. -1000.    -6.    -8.   -10.   -12.   -14.   -16.   -18.   -20.]
 [-1000. -1000.    -6.    -8.   -10.   -12.   -14.   -16.   -18.   -20.]
 [-1000. -1000.    -6.    -8.   -10.   -12.   -14.   -16.   -18.   -20.]
 [-1000. -1000.    -8.   -10.   -12.   -14.   -16.   -18.   -20.   -22.]
 [-1000. -1000.   -10.   -12.   -14.   -16.   -18.   -20.   -22.   -24.]
 [-1000.   -10.   -12.   -14.   -16.   -18.   -20.   -22.   -24.   -26.]]
Current State: explore
[[   -9.   -11.   -13.   -15.   -17.   -19.   -21.   -23.   -25.   -27.]
 [   -7.    -9.   -11.   -13.   -15.   -17.   -19.   -21.   -23.   -25.]
 [   -5.    -7.    -9.   -11.   -13.   -15.   -17.   -19.   -21.   -23.]
 [-

2it [00:00,  4.38it/s]

...


Current State: explore
[[  -20.   -18.   -16.   -14.   -12.   -10.   -10.   -10.   -12.   -14.]
 [  -18.   -16.   -14.   -12.   -10.    -8.    -8.    -8.   -10.   -12.]
 [  -16.   -14.   -12.   -10.    -8.    -6.    -6.    -6.    -8.   -10.]
 [  -14.   -12.   -10.    -8.    -6.    -2. -1000. -1000.    -6.    -8.]
 [  -12.   -10.    -8.    -6. -1000. -1000. -1000. -1000. -1000.    -6.]
 [  -14.   -12.   -10.    -8.    -6. -1000. -1000. -1000.    -6.    -8.]
 [  -16.   -14.   -12.   -10.    -8.    -6.    -6.    -6.    -8.   -10.]
 [  -18.   -16.   -14.   -12.   -10.    -8.    -8.    -8.   -10.   -12.]
 [  -20.   -18.   -16.   -14.   -12.   -10.   -10.   -10.   -12.   -14.]
 [  -22.   -20.   -18.   -16.   -14.   -12.   -12.   -12.   -14.   -16.]]
Current State: return_to_hive
Current State: return_to_hive
Current State: return_to_hive
Current State: explore
[[  -22.   -20.   -18.   -16.   -14.   -12.   -10.    -8.   -10.   -12.]
 [  -20.   -18.   -16.   -14.   -12.   -10.    -8.    -6.  

5it [00:01,  4.60it/s]

Current State: explore
[[   -9.    -7.    -7.    -7.    -9.   -11.   -13.   -15.   -17.   -19.]
 [-1000. -1000. -1000. -1000.    -7.    -9.   -11.   -13.   -15.   -17.]
 [-1000. -1000. -1000. -1000. -1000.    -7.    -9.   -11.   -13.   -15.]
 [-1000. -1000. -1000. -1000.    -5.    -7.    -9.   -11.   -13.   -15.]
 [-1000. -1000.    -5.    -5.    -7.    -9.   -11.   -13.   -15.   -17.]
 [   -9.    -7.    -7.    -7.    -9.   -11.   -13.   -15.   -17.   -19.]
 [  -11.    -9.    -9.    -9.   -11.   -13.   -15.   -17.   -19.   -21.]
 [  -13.   -11.   -11.   -11.   -13.   -15.   -17.   -19.   -21.   -23.]
 [  -15.   -13.   -13.   -13.   -15.   -17.   -19.   -21.   -23.   -25.]
 [  -17.   -15.   -15.   -15.   -17.   -19.   -21.   -23.   -25.   -27.]]
Current State: explore
[[  -10.    -8.    -6.    -6.    -8.   -10.   -12.   -14.   -16.   -18.]
 [-1000. -1000. -1000. -1000.    -6.    -8.   -10.   -12.   -14.   -16.]
 [-1000. -1000. -1000. -1000. -1000.    -6.    -8.   -10.   -12.   -14.]
 [-1

6it [00:01,  4.69it/s]


Current State: explore
[[-1000.    -6.    -8.   -10.   -12.   -14.   -16.   -18.   -20.   -22.]
 [-1000. -1000.    -6.    -8.   -10.   -12.   -14.   -16.   -18.   -20.]
 [-1000. -1000.    -6.    -8.   -10.   -12.   -14.   -16.   -18.   -20.]
 [-1000. -1000.    -6.    -8.   -10.   -12.   -14.   -16.   -18.   -20.]
 [-1000.    -6.    -8.   -10.   -12.   -14.   -16.   -18.   -20.   -22.]
 [   -6.    -8.   -10.   -12.   -14.   -16.   -18.   -20.   -22.   -24.]
 [   -8.   -10.   -12.   -14.   -16.   -18.   -20.   -22.   -24.   -26.]
 [  -10.   -12.   -14.   -16.   -18.   -20.   -22.   -24.   -26.   -28.]
 [  -12.   -14.   -16.   -18.   -20.   -22.   -24.   -26.   -28.   -30.]
 [  -14.   -16.   -18.   -20.   -22.   -24.   -26.   -28.   -30.   -32.]]
Current State: explore
[[-1000.    -7.    -9.   -11.   -13.   -15.   -17.   -19.   -21.   -23.]
 [-1000. -1000.    -7.    -9.   -11.   -13.   -15.   -17.   -19.   -21.]
 [-1000. -1000.    -7.    -9.   -11.   -13.   -15.   -17.   -19.   -21.]
 [-

In [6]:
model_data

,max_bee_energy,Run,Bee energy,Nectar Collected,Nectar stored,Nectar/T,height,width,init_bees,init_flowers,init_min_nectar_grade,init_max_nectar_grade,min_nectar,max_nectar,nectar_respawn_interval,collect_negative_value_nectar,perception_range
0,30,0,0,70,0,0.583333,10,10,1,6,1,30,1,1,50,True,1
1,30,1,0,26,0,0.342105,10,10,1,6,1,30,1,1,50,True,1
2,30,2,0,16,0,0.242424,10,10,1,6,1,30,1,1,50,True,1
3,40,3,0,59,0,0.495798,10,10,1,6,1,30,1,1,50,True,1
4,40,4,0,24,0,0.285714,10,10,1,6,1,30,1,1,50,True,1
5,40,5,0,38,0,0.387755,10,10,1,6,1,30,1,1,50,True,1
